# This notepad demonstrates the usage Open AI's LLM model and Langgraph Agent Framework for performing sentiment analysis on financial news articles.

### Install required packages

In [1]:
!pip install -q langgraph langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.4 MB/s eta 0:00:00


### Import required modules

In [2]:
import os
from google.colab import userdata
from langgraph.graph import StateGraph, START,END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from typing import Dict, List, TypedDict, Any

### Get the api_key and initilize the model

In [3]:
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [61]:
model=ChatOpenAI(temperature=0, model='gpt-4o-mini')

### Define a class with the required variables for the graph

In [62]:
class newsState(TypedDict):
  news: str
  news_sentiment: str
  user_output: str
  messages: List[Dict[str,Any]]

### Define a function for performing sentiment analysis on the input news article for the graphs node.

In [56]:
def perform_sentiment_analysis(newsState):
  """Analyze financial news and perform sentiment analysis."""

  prompt="""You are a financial news sentiment analysis expert. Analyze the sentiment of the following financial news article and classify it as either positive, negative, or neutral.

  {newsState['news']}

  Respond with the sentiment only. Do not explain your reasoning."""

  messages=[HumanMessage(content=prompt)]

  response = model.invoke(messages)

  new_messages=[{'role':'user', 'content' : prompt}, {'role':'assistant', 'content':response.content}]

  return {'news_sentiment' : response.content, 'messages' : new_messages}

### Define a function for providing the sentiment as an output.

In [57]:
def draft_user_response(newsState):
  """Draft a user response based on the sentiment analysis."""
  output_msg=f"Thanks for asking! \n\nHere are the details: \n\nNews Article: {newsState['news']} \n\nSentiment: {newsState['news_sentiment']} \n\nHope you are happy with the response."

  return {'user_output' : output_msg}

### Define the graph with the required nodes and edges

In [58]:
newsGraph=StateGraph(newsState)
newsGraph.add_node("perform_sentiment_analysis",perform_sentiment_analysis)
newsGraph.add_node("draft_user_response",draft_user_response)
newsGraph.add_edge(START,"perform_sentiment_analysis")
newsGraph.add_edge("perform_sentiment_analysis","draft_user_response")
newsGraph.add_edge("draft_user_response",END)
compiled_graph=newsGraph.compile()

### Verify the output

In [59]:
input_news="""Stock Radar: LTIMindtree gives a breakout from falling wedge pattern; time to buy after over 30% fall from highs.
LTIMindtree Ltd, after a 32% fall, shows signs of a bullish comeback, breaking out from a falling wedge pattern.
Experts suggest short-term traders can buy with a target of Rs 4,820 in 3-4 weeks, supported by positive technical indicators like Supertrend buy and RSI divergence.
A stop loss at Rs 4,300 is recommended."""
output_news_sentiment=compiled_graph.invoke({'news':input_news})
print(output_news_sentiment['user_output'])

Thanks for asking! 

Here are the details: 

News Article: Stock Radar: LTIMindtree gives a breakout from falling wedge pattern; time to buy after over 30% fall from highs. 
LTIMindtree Ltd, after a 32% fall, shows signs of a bullish comeback, breaking out from a falling wedge pattern. 
Experts suggest short-term traders can buy with a target of Rs 4,820 in 3-4 weeks, supported by positive technical indicators like Supertrend buy and RSI divergence. 
A stop loss at Rs 4,300 is recommended. 

Sentiment: Neutral 

Hope you are happy with the response.


In [60]:
input_news="""SEBI accuses Adani nephew in insider trading case, he seeks to settle. India's markets regulator has alleged Pranav Adani,
director of several Adani group companies and the nephew of the billionaire founder, shared price sensitive information and breached regulations aimed at preventing insider trading,
according a document reviewed by Reuters."""
output_news_sentiment=compiled_graph.invoke({'news':input_news})
print(output_news_sentiment['user_output'])

Thanks for asking! 

Here are the details: 

News Article: SEBI accuses Adani nephew in insider trading case, he seeks to settle. India's markets regulator has alleged Pranav Adani, 
director of several Adani group companies and the nephew of the billionaire founder, shared price sensitive information and breached regulations aimed at preventing insider trading, 
according a document reviewed by Reuters. 

Sentiment: Neutral 

Hope you are happy with the response.
